In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e12/sample_submission.csv
/kaggle/input/playground-series-s4e12/train.csv
/kaggle/input/playground-series-s4e12/test.csv


# 1) Data Cleaning

## configuring training data

In [2]:
train_data = pd.read_csv('/kaggle/input/playground-series-s4e12/train.csv')
y = train_data['Premium Amount']
train_data = train_data.drop(['id', 'Premium Amount'], axis=1)

# function to check for NaN values in a specified column
def checknan(colname):
    return train_data[train_data[colname].isna()]

# function to fillna with either mean, median, 0, or another column
def fillNaN(colnamelist, type, othercol=None):
    if type == 'mean':
        for colname in colnamelist:
            train_data.fillna({colname: train_data[colname].median()}, inplace = True)
    elif type == 'median':
        for colname in colnamelist:
            train_data.fillna({colname: train_data[colname].median()}, inplace = True)
    elif type == 'mode':
        for colname in colnamelist:
            train_data.fillna({colname: train_data[colname].mode()}, inplace = True)
    elif type == 0:
        for colname in colnamelist:
            train_data.fillna({colname: 0}, inplace = True)
    elif type == 'othercol':
        for colname in colnamelist:
            train_data.fillna({colname: train_data[othercol]}, inplace = True)
    print(f'fillNaN [{type}] complete')

nan_cols = train_data.columns[train_data.isna().any()]
for col in nan_cols:
    print(f'- {col}')

# fillna for column: Age
fillNaN(['Age'], 'mean')
# fillna for column: Annual Income
fillNaN(['Annual Income'], 'median')
# fillna for column: Marital Status
fillNaN(['Marital Status'], 'mode')
# fillna for column: Number of Dependents
fillNaN(['Number of Dependents'], 'mean')
# fillna for column: Occupation
fillNaN(['Occupation'], 'mode')
# fillna for column: Health Score
fillNaN(['Health Score'], 'mean')
# fillna for column: Previous Claims
fillNaN(['Previous Claims'], 'mode')
# fillna for column: Vehicle Age
fillNaN(['Vehicle Age'], 'median')
# fillna for column: Credit Score
fillNaN(['Credit Score'], 'median')
# fillna for column: Insurance Duration
fillNaN(['Insurance Duration'], 'median')
# fillna for column: Customer Feedback
fillNaN(['Customer Feedback'], 'mode')



- Age
- Annual Income
- Marital Status
- Number of Dependents
- Occupation
- Health Score
- Previous Claims
- Vehicle Age
- Credit Score
- Insurance Duration
- Customer Feedback
fillNaN [mean] complete
fillNaN [median] complete
fillNaN [mode] complete
fillNaN [mean] complete
fillNaN [mode] complete
fillNaN [mean] complete
fillNaN [mode] complete
fillNaN [median] complete
fillNaN [median] complete
fillNaN [median] complete
fillNaN [mode] complete
